# Data Analaysis 
This notebook will contain the bulk of the analysis done to identify the model which best predicts the Material Group a composite waste item belongs to. The stages in the analysis include:
1. Data understanding
2. Data preparation
3. Feature Analysis, Train / Test split, Baseline Model
4. Modelling
5. Iteration and Improvements
6. Conclusions

In [1]:
# Begin by importing the necessary libraries 

In [2]:
!pip3 install pandas
!pip3 install plotly
!pip3 install numpy
!pip3 install seaborn
!pip3 install matplotlib.pyplot
!pip3 install mpl_toolkits
!pip3 install sklearn
!pip3 install statsmodels
!pip3 install random
!pip3 install time


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Usin


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.compose import make_column_transformer

import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from random import shuffle, seed

In [4]:
data = pd.read_csv('data/nyc-waste-data.csv')
data.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper,Designated Paper,Citywide


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832 entries, 0 to 1831
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Material                         1832 non-null   object 
 1   Aggregate Percent                1832 non-null   float64
 2   Refuse Percent                   1832 non-null   float64
 3   MGP Percent                      1603 non-null   float64
 4   Paper Percent                    1603 non-null   float64
 5   Organic Percent                  1145 non-null   float64
 6   Material Group                   1832 non-null   object 
 7   DSNY Diversion Summary Category  1832 non-null   object 
 8   Location                         1832 non-null   object 
dtypes: float64(5), object(4)
memory usage: 128.9+ KB


## Data Understanding 
In this stage, understand the meaning of the various rows and columns in this table. 

* `Material`: The classification of all materials in the waste stream into categories for sorting, weighing, and counting purposes.
* `Aggregate Percent`: Percentage of a material in all combined applicable streams.
* `Refuse Percent`: Percentage of a material in the refuse stream.
* `MGP Percent`: Percentage of a material in the metal, glass, and plastic stream.
* `Paper Percent`: Percentage of a material in the paper stream.
* `Organic Percent`: Percentage of a material in the organics stream.
* `Material Group`: Groupings of material categories used to simplify or generalize results.
* `DSNY Diversion Summary Category`: Groupings of material categories based on how the material is collected and diverted, according to DSNY designations.
* `Location` : Geographic indicator of a materials percentage to each waste stream. Location also includes a distinction for materials collected in schools and NYCHA citywide.

While looking at the output of `data.head()` above we can see that the value of "Location" may not be relevant to this analysis as it specifies where the item was collected. This is not relevant in categorizing an item.

## Data Preparation 

This stage will include data cleaning. This includes identifying necessary data for analysis and removing uneeded data that is either not relevant to the research question. This stage also includes preparing the target data and test/train data.  

In [6]:
# Cleaning the data 

# Remove any empty values  
data.dropna()

# Remove the Location column as that is not relevant to the analysis we will be performing 
cleandata = data
columns_to_drop = ['Location', 'DSNY Diversion Summary Category']

# Remove the DSNY Diversion Summary Category column as we are targeting a different 
# categorization (the "Material Group" category)
cleandata = cleandata.drop(columns_to_drop, axis=1)

# For clarity we can rename "Material" to "Item Name" and "Material Group" --> to "Category"
new_column_names = {'Material': 'Item Name', 'Material Group': 'Category'}
cleandata.rename(columns=new_column_names, inplace=True)

# Now we can proceed with the analysis
cleandata.head()

,Item Name,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Category
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper


In [7]:
# Set target column and data to later split into training and testing data
X = cleandata.drop('Category', axis = 1)
y = cleandata["Category"]

## Feature Analysis, Train / Test split, Baseline Model
Feature analysis will consist of separating the features into numerical and categorical data, then creating transformers so that the features can be processed together to train then test the models. 

Train/Test split will be done of the cleandata so that we do not bias the model and have separate datasets to train and a separate dataset to test. We will use a test size of 0.3 and `randomstate=22`.

Then, we will create a baseline model to use as a point of comparison to see the frequently of each material category.

In [8]:
# Separate the numerical and categorical columns
numerical_cols=['Aggregate Percent', 'Refuse Percent', 'MGP Percent', 'Paper Percent', 'Organic Percent']
categorical_cols=['Item Name']

numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])


In [9]:
# Split the data into training and testing data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

In [10]:
# View the baseline for all cateogry types, which confirms that (after normalization) plastic is the highest 
# counted category - this will serve as a good reference point for future model analysis
baseline = cleandata['Category'].value_counts(normalize = True)
print("baseline for all materials: ", baseline)

baseline for all materials:  Category
Plastic                     0.711790
Paper                       0.069869
Special Waste               0.056769
Organic                     0.052402
Metal                       0.043668
Glass                       0.026201
E-Waste                     0.021834
C&D                         0.013100
Miscellaneous Inorganics    0.004367
Name: proportion, dtype: float64


## Modelling

Now, we will test various model types. To keep track of key metrics that will be valuable in comparison we will use a model comparisons dataframe that tracks: the test time, training accuracy, and testing data accuracy. The models that will be tested are: 
* Logistic Regression
* KNN (with multiple N values) 
* Decision Tree (with multiple max depth values)
* SVM (with different kernel types) 

Cross validation will also be used to further refine models. 

In [61]:
# Start by creating the df to store metrics on all models 
model_comparisons_dict = {'Model':[], 
        'Train Time':[],
        'Train Accuracy':[],
        'Test Accuracy':[]
       }
model_comparisons = pd.DataFrame(model_comparisons_dict)

In [62]:
# Model 1: Logistic Regression attempt 
model = LogisticRegression(max_iter=10000, random_state=42)
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('logreg', model)
])

log_reg_start_time = time.time()
pipe.fit(X_train, y_train)
log_reg_end_time = time.time()

log_reg_train_time = log_reg_end_time - log_reg_start_time
print("Train time for logistic regression: ", log_reg_train_time)

Train time for logistic regression:  0.07974505424499512


In [63]:
preds = pipe.predict(X_test)
# Evaluate the accuracy 
log_reg_test_accuracy = accuracy_score(y_test, preds)
log_reg_train_accuracy = accuracy_score(y_train, pipe.predict(X_train))

print("accuracy score (train): ", log_reg_train_accuracy)
print("accuracy score (test): ", log_reg_test_accuracy)

model_comparisons.loc[len(model_comparisons.index)] = ['Logistic Regression', log_reg_train_time, log_reg_train_accuracy, log_reg_test_accuracy] 

accuracy score (train):  0.983619344773791
accuracy score (test):  0.9472727272727273


In [64]:
# Model 2: KNN with n = 5 default

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier(n_neighbors=5))])

# Fitting the pipeline and measuring time 
knn_start_time = time.time()
knn_pipeline.fit(X_train, y_train)
knn_end_time = time.time()

# Collect metrics 
knn_train_time = knn_end_time - knn_start_time; 
print("Train time for KNN: ", knn_train_time)

preds = pipe.predict(X_test)

Train time for KNN:  0.01750636100769043


In [65]:
# Evaluate the accuracy 
knn_train_accuracy = accuracy_score(y_train, pipe.predict(X_train))
knn_test_accuracy = accuracy_score(y_test, preds)
print("knn accuracy score (train): ", knn_train_accuracy)
print("knn accuracy score (test): ", knn_test_accuracy)
model_comparisons.loc[len(model_comparisons.index)] = ['KNN (n = 5)', knn_train_time, knn_train_accuracy, knn_test_accuracy] 

knn accuracy score (train):  0.983619344773791
knn accuracy score (test):  0.9472727272727273


In [66]:
# Model 3: Decision Tree with max depth = 3
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=3))
])
# Fitting the pipeline and measuring time 
dt_start_time = time.time()
dt_pipeline.fit(X_train, y_train)
dt_end_time = time.time()

# Collect metrics 
dt_train_time = dt_end_time - dt_start_time; 
dt_accuracy_test = dt_pipeline.score(X_test, y_test)
dt_accuracy_train = dt_pipeline.score(X_train, y_train)

print("Decision Tree Train Time: ", dt_train_time)
print("Decision Tree Train Accuracy: ", dt_accuracy_train)
print("Decision Tree Test Accuracy: ", dt_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['Decision Tree (depth = 3)', dt_train_time, dt_accuracy_train,dt_accuracy_test]


Decision Tree Train Time:  0.01891922950744629
Decision Tree Train Accuracy:  0.781591263650546
Decision Tree Test Accuracy:  0.7472727272727273


In [67]:
# Model 4: SVM 
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear'))
])

# Fitting the pipeline  and measuring time 
svm_start_time = time.time()
svm_pipeline.fit(X_train, y_train)
svm_end_time = time.time()

# Collect metrics 
svm_train_time = svm_end_time - svm_start_time; 
svm_accuracy_test = svm_pipeline.score(X_test, y_test)
svm_accuracy_train = svm_pipeline.score(X_train, y_train)

print("SVM Train Time: ", svm_train_time)
print("SVM Train Accuracy: ", svm_accuracy_train)
print("SVM Test Accuracy: ", svm_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['SVM (linear)', svm_train_time, svm_accuracy_train, svm_accuracy_test]
                                                       

SVM Train Time:  0.09536623954772949
SVM Train Accuracy:  1.0
SVM Test Accuracy:  1.0


## Iteration and Improvements 
There are a couple of key observations from the model results above. 
* The KNN model was only tested with one value of N, but seems to be the most performant. We can experiment with more N values to see if that improves the accuracy or runtime of the model. 
* The Decision Tree was only tested with a model of max depth = 3, so we may similarly want to experiment with different depths 
* The SVM model has an accuracy of 1.0 - this either means the model is overfitted or requires hyperparameter tuning. To explore this, we will perform K-Fold Cross Validation and GridSearchCV. 

In [68]:
# KNN Model (iteration): Trying different N values 
n_values = [3, 5, 10, 20, 50]

for n in n_values: 
    knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('knn', KNeighborsClassifier(n_neighbors=n))])

    # Fitting the pipeline and measuring time 
    knn_start_time = time.time()
    knn_pipeline.fit(X_train, y_train)
    knn_end_time = time.time()
    
    knn_accuracy_test = knn_pipeline.score(X_test, y_test)
    knn_accuracy_train = knn_pipeline.score(X_train, y_train)
    knn_train_time = knn_end_time - knn_start_time; 
    
    model_name = "KNN (n = " + str(n) + ")"

    print(model_name + "Train Time : ", knn_train_time)
    print(model_name + "Train Accuracy: ", knn_accuracy_train)
    print(model_name + "Test Accuracy: ", knn_accuracy_test)

    # Collect metrics
    
    model_comparisons.loc[len(model_comparisons.index)] = [model_name, knn_train_time, knn_accuracy_train, knn_accuracy_test]     

KNN (n = 3)Train Time :  0.017691373825073242
KNN (n = 3)Train Accuracy:  1.0
KNN (n = 3)Test Accuracy:  1.0
KNN (n = 5)Train Time :  0.015609264373779297
KNN (n = 5)Train Accuracy:  0.9984399375975039
KNN (n = 5)Test Accuracy:  0.9945454545454545
KNN (n = 10)Train Time :  0.014902114868164062
KNN (n = 10)Train Accuracy:  0.9695787831513261
KNN (n = 10)Test Accuracy:  0.9272727272727272
KNN (n = 20)Train Time :  0.015343427658081055
KNN (n = 20)Train Accuracy:  0.8361934477379095
KNN (n = 20)Test Accuracy:  0.8163636363636364
KNN (n = 50)Train Time :  0.014756917953491211
KNN (n = 50)Train Accuracy:  0.7706708268330733
KNN (n = 50)Test Accuracy:  0.7472727272727273


In [69]:
# Decision Tree Iteration: increase depth size
depths = [5, 10, 50, 100]

for depth in depths:
    dt_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(max_depth=depth))
    ])
    # Fitting the pipeline and measuring time 
    dt_start_time = time.time()
    dt_pipeline.fit(X_train, y_train)
    dt_end_time = time.time()

    # Collect metrics 
    dt_train_time = dt_end_time - dt_start_time; 
    dt_accuracy_test = dt_pipeline.score(X_test, y_test)
    dt_accuracy_train = dt_pipeline.score(X_train, y_train)
    
    model_name = "Decision Tree (depth = " + str(depth) + ")"

    print(model_name + "Train Time: ", dt_train_time)
    print(model_name + "Train Accuracy: ", dt_accuracy_train)
    print(model_name + "Test Accuracy: ", dt_accuracy_test)
    
    # Add results to table
    model_comparisons.loc[len(model_comparisons.index)] = [model_name, dt_train_time, dt_accuracy_train,dt_accuracy_test]

Decision Tree (depth = 5)Train Time:  0.020813941955566406
Decision Tree (depth = 5)Train Accuracy:  0.8213728549141965
Decision Tree (depth = 5)Test Accuracy:  0.7654545454545455
Decision Tree (depth = 10)Train Time:  0.018789291381835938
Decision Tree (depth = 10)Train Accuracy:  0.8861154446177847
Decision Tree (depth = 10)Test Accuracy:  0.8
Decision Tree (depth = 50)Train Time:  0.021295547485351562
Decision Tree (depth = 50)Train Accuracy:  1.0
Decision Tree (depth = 50)Test Accuracy:  0.9381818181818182
Decision Tree (depth = 100)Train Time:  0.02162337303161621
Decision Tree (depth = 100)Train Accuracy:  1.0
Decision Tree (depth = 100)Test Accuracy:  0.9381818181818182


In [70]:
# SVM Model - 5 fold cross-validation object
folds = [5, 10, 50, 100]
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear'))
])

for fold in folds:
    kfold = KFold(n_splits=fold, shuffle=True, random_state=42)
    accuracy_scores = cross_val_score(svm_pipeline, X, y, cv=kfold, scoring='accuracy')
#     print("Accuracy scores for each fold:", accuracy_scores)

    mean_accuracy = accuracy_scores.mean()
    std_accuracy = accuracy_scores.std()
        
    fold_str = "Fold = " + str(fold) + " "
    print(fold_str+ f"Mean accuracy: {mean_accuracy:.4f}")

Fold = 5 Mean accuracy: 1.0000
Fold = 10 Mean accuracy: 1.0000
Fold = 50 Mean accuracy: 1.0000
Fold = 100 Mean accuracy: 1.0000


In [71]:
# May be too simplistic, one last attempt with GridSearchCV for SVM pipeline using 
# more parameters to tune the model
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svm', SVC(kernel='poly'))
])

param_grid = {
    'svm__C': [0.1, 1, 10],  # SVM regularization parameter
    'svm__kernel': ['linear', 'rbf'],  # SVM kernel types
    'svm__gamma': [0.1, 1, 10]  # SVM kernel coefficient for RBF kernel
}

grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X, y)

# Get the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

# We can conclude that the data is too simplistic for this model

Best parameters: {'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'linear'}
Best accuracy: 1.0


## Conclusions 
Now we will look at all the models to compare accuracies and runtime to identify the most accurate model.

In [72]:
model_comparisons.head(20)

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.079745,0.983619,0.947273
1,KNN (n = 5),0.017506,0.983619,0.947273
2,Decision Tree (depth = 3),0.018919,0.781591,0.747273
3,SVM (linear),0.095366,1.000000,1.000000
4,KNN (n = 3),0.017691,1.000000,1.000000
5,KNN (n = 5),0.015609,0.998440,0.994545
6,KNN (n = 10),0.014902,0.969579,0.927273
7,KNN (n = 20),0.015343,0.836193,0.816364
8,KNN (n = 50),0.014757,0.770671,0.747273
9,Decision Tree (depth = 5),0.020814,0.821373,0.765455


In [74]:
# Plot model against test accuracy 
fig = px.scatter(model_comparisons, x='Model', y='Test Accuracy',
                 title='Model Performance: Test Accuracy')
fig.show()

fig2 = px.scatter(model_comparisons, x='Model', y='Train Time',
                 title='Model Performance: Train Time')
fig2.show()

We can make some high-level conclusions about the models: 
* The fastest train time was 0.014757s for the KNN N=50 model, closely followed by N=10 at 0.014902, then N=5 at 0.
* The highest test accuracy was the SVM model with a 1.0 accuracy 
* The second highest accuracy was the KNN model at N=5 with 0.994545
* We can notice overfitting in the KNN models when the model is at N=3, and then a max accuracy at K=5, which steadily decreases as K increases 
* We can notice a max accuracy for the Decision Tree at higher depths, which also may be overfitting (since the train accuracy is 1.0). 

The next notebook `results.ipynb` will present the results from these observations and propose applications of these results as well as next steps. 

In [75]:
%store model_comparisons

Stored 'model_comparisons' (DataFrame)
